## Notebook for producing vOTU table from raw sequencing reads as in "Minnesota peat viromes reveal terrestrial and aquatic niche partitioning for local and global viral populations"
Anneliek ter Horst


## QC reads with trimmomatic

- Trim the reads with trimmomatic, following Roux et al., 2017
- minimum base quality threshold of 30 evaluated on sliding windows of 4 bases, and minimum read length of 50
- As a rule of thumb newer libraries will use TruSeq3, but this really depends on your service provider.
- http://www.usadellab.org/cms/?page=trimmomatic

In [ ]:
# load modules
module load java
module load trimmomatic

# use trimmomatic to trim adapters, make sure you have the correct adapterfile
trimmomatic PE -threads 8 -phred33 $1 ${1%%_1*}_2.fastq.gz \
../trimmed/${1%%_1*}_1_trimmed.fq.gz unpaired/${1%%_1*}_1_Unpaired.fq.gz \
../trimmed/${1%%_1*}_2_trimmed.fq.gz unpaired/${1%%_1*}_2_Unpaired.fq.gz \
ILLUMINACLIP:/adapters/TruSeq3-PE.fa:2:30:10 \
SLIDINGWINDOW:4:30 MINLEN:50 

## Remove PhiX174 with bbduk
- Phix174 is a phage sequence that is used to aid in sequencing. This needs to be removed from our sequencing data
- https://jgi.doe.gov/data-and-tools/bbtools/bb-tools-user-guide/bbduk-guide/

In [ ]:
# load module
module load java
module load bbmap

# use bbduk to remove phix contamination
bbduk.sh in1=$1 in2=${1%%_1*}_2_trimmed.fq.gz \
out1=../remove_phix/${1%%_1*}_1_trimmed_bbduk.fq.gz out2=../remove_phix/${1%%_1*}_2_trimmed_bbduk.fq.gz \
ref=/bbduk/phix_genome.fa k=31 \
hdist=1 stats=../remove_phix/${1%%}_stats.txt

## Assembly with MEGAHIT
- Assemble viromes following Roux et al., 2017:

- Assemble total soil metagenomes following Sczyra et al., 2017 and Vollmer et al., 2017 

- https://github.com/voutcn/megahit

In [ ]:
# MEGAHIT for single sample assembly
megahit -1 $1 -2 ${1%%1_R1*}1_R2_trimmed_bbduk.fq.gz -o ../assemblies_persample/${1%%_R1*} \
--out-prefix ${1%%_R1*} --min-contig-len 10000 --continue


# Co-assemly viromes
READ1=*_R1_*
READ2=*_R2_*
READ1s=`ls $READ1 | python -c 'import sys; print ",".join([x.strip() for x in sys.stdin.readlines()])'`
READ2s=`ls $READ2 | python -c 'import sys; print ",".join([x.strip() for x in sys.stdin.readlines()])'`

megahit -1 ${READ1s} -2 ${READ2s} -o co_assembly_virome -t 64 -m 480e9 --mem-flag=2 \
--k-min 27 --out-prefix co_assembly_virome \
--min-contig-len 10000 --presets meta-large


# total soil metagenomes
megahit -1 $1 -2 ${1%%1_R1*}1_R2_trimmed_bbduk.fq.gz -o ../bact_assemblies_persample/${1%%_R1*} \
--out-prefix ${1%%_R1*} --min-contig-len 2000

## DeepVirfinder
- Only contigs that have a virFinder score => 0.9 and p <0.05 Gregory et al., 2019
- https://github.com/jessieren/DeepVirFinder

In [ ]:
# run dvf
python dvf.py -i $1 -l 10000 -o ${1%%.fna*}_virfinder -c 8 \

# use custom script to parse dvf data
python parse_DVF.py $f/*.txt ${f%%_virfinder*}_seq_names.txt


# Pull those sequences from the fasta files
for f in *.fa; do
awk -F'>' 'NR==FNR{ids[$0]; next} NF>1{f=($2 in ids)} f' \
../virfinder/${f%%.fa*}_seq_names.txt $f \
> ../virfinder/${f%%contigs.fa*}virfinder.fa
done

In [ ]:
"""parse_dvf.py, Anneliek ter Horst, September 2019
This script filters the deepVirfinder output to only keep entries with a
virfinder score > 0.9 and a pvalue < 0.05"""

#! /usr/bin/env python 
# imports
import pandas as pd
import numpy as np
import re
import sys


# open the coverage table
df = pd.read_csv(sys.argv[1] ,sep='\t')

df = df[df.score > 0.899]
df = df[df.pvalue < 0.05]
print(len(df))

df = df.name
df.to_csv(sys.argv[2], sep='\t',index=False)

In [ ]:
# pull the sequences that have high enough score from the fasta file with all contigs
for f in *.fna; do
awk -F'>' 'NR==FNR{ids[$0]; next} NF>1{f=($2 in ids)} f' \
${f%%.fna*}_seq_names.txt $f > ${f%%contigs.fna*}virfinder.fa
done

## VirSorter
- Use Virsorter as well to predict viral sequences
- Take only virsorter sequences from category 1,2,4 and 5
- https://github.com/simroux/VirSorter

In [ ]:
# run virsorter
wrapper_phage_contigs_sorter_iPlant.pl -f $f -ncpu 8 \
--data-dir virsorter-data/ -db 1 --wdir ${f%%.contigs.fa*}_virsorter --virome

In [ ]:
# Concatenate all virSorter output in said categories
for f in *_virsorter; do
cat $f/Pr*/VIRSorter*cat-[1245].fasta >> $f/${f%%_virsorter*}_virsort_viralseqs.fa
done


# Concatenate, rename and sort viral sequences with bbmap
- concatenate findings from Virsorter and Virfinder
- rename those sequences according to the dataset they came from
- Sort on lenght so we can cluster after
- https://jgi.doe.gov/data-and-tools/bbtools/bb-tools-user-guide/bbmap-guide/

In [ ]:
# concatenate
cat *_virsorter_seqs.fa *_virfinder.fa >> *_all_found_viral_seqs.fa

# rename (with bbmap rename.sh)
rename.sh in=*_all_found_viral_seqs.fa out=*_all_found_viral_rename.fa prefix=SPRUCE_virome_

# sort on length (with bbmap sortbyname.sh)
sortbyname.sh in=*_all_found_viral_rename.fa  out=all_viral_seqs.sorted.fa length descending

# Clustering found sequences with CD-hit
- Clustering sequences at 95% ANI over 85% of the sequence, relative to the shorter sequence, following Roux et al., 2019
- https://github.com/weizhongli/cdhit/wiki/2.-Installation


In [ ]:
# use CD-hit to cluster found viral sequences
cd-hit-est -i all_viral_seqs.sorted.fa -o all_viral_seqs.sorted.cluster.fa \
-c 0.95 -aS 0.85 -M 7000 -T 10

# use CD-hit to cluster those viral sequences with PIGEON
cd-hit-est-2d -i all_viral_seqs.sorted.cluster.fa \
-i2 PIGEONv1.0.sorted.fa \
-o unique_viral_seqs.fa \
-c 0.95 -aS 0.85 -M 20000 -T 10


# make one database from PIGEON and the newly found unique viral sequences
cat PIGEONv1.0.sorted.fa unique_viral_seqs.fa >> PIGEON_and_new_SPRUCE.fa


# Mapping back with BBMAP and keep only contigs that are > 75% covered in lenght
- Use bbmap to map back clean reads to PIGEON database to see what viruses can be recovered via read mapping
- Do this at 90% nucleotide identity, following Roux et al., 2017 
- Do this at 75% of the contig lenght, following Roux et al., 2017
- https://jgi.doe.gov/data-and-tools/bbtools/bb-tools-user-guide/bbmap-guide/

In [ ]:
# produce reference mapping file
bbmap.sh ref=PIGEON_and_new_SPRUCE.fa

# map reads to the reference file (paired end reads)
bbmap.sh in1=$f in2=${f%%_R1_*}_R2_bbduk.fq.gz \
out=${f%%_R1_*}.sam \
minid=0.90 threads=25

# make bam files from sam files
samtools view -F 4 -bS file.sam | samtools sort > file_sortedIndexed.bam

# index the bam file
samtools index ${f%%R1_*}_sortedIndexed.bam

# Calculate coverage length using bedtools
- calculate for each bam file the coverage length
- bga makes sure that also regions with 0 coverage are reported
- max reports all reads with depth >10 as 10. Is faster. 
- https://bedtools.readthedocs.io/en/latest/


In [ ]:
# run bedtools
bedtools genomecov -ibam $f -bga -max 10 > ${f%.bam*}.tsv

# use custom python script to filter out contigs that aren't covered > 75%
for f in $(ls *.tsv); do python parse_bedtools.py $f ${f%%.tsv*}_under75.tsv ; done; 

# for each sample make a list of names that are not 75% covered
for f in $(ls *75.tsv); do cat $f | cut -d$'\t' -f2  >  ${f%%.tsv*}.txt ; done;

# coverage table needs to be filtered based on the contigs that dont have 75% coverage for each sample
# so make a csv with [header]=filename, thus will correspond to a name in the covtab
for filename in $(ls *.txt)
do
	sed "1s/^/${filename} \n/" ${filename} > $filename.new 
	echo Done ${filename} 
done

# move all these files into folders
mkdir under_75_csv 
mv *75.tsv under_75_csv 
mkdir under_75_txt
mv *75.txt under_75_txt
mkdir under_75_name_added
mv *.new under_75_name_added

# make one giant file that contains what contig in which sample isn't covered 75%
paste -d "|" *txt.new > ../SPRUCE_contigs_under75.csv

In [ ]:
#! /usr/bin/env python 

'''
python parse_bedtools.py, Anneliek ter Horst, September 2019
bedtools genomecov outputs a tsv that we will manipulate with pandas to 
obtain the average coverage per contig. If its more than 75%, we will discard the contig.
'''

# imports
from __future__ import division
import sys
import pandas as pd

# open the file to read with pandas
df = pd.read_csv(sys.argv[1],sep='\t', header=None)

# add column names so we know what we are looking at
df.columns = ['sequence', 'start_pos', 'stop_pos' , 'coverage']

# Add a column with the number of nucleotides that share the same coverage
df['num_of_nucl'] = df['stop_pos'] - df['start_pos']

# for each sequence, add a extra column witht the sequences total length. Calculating percentages
# becomes easier this way
df['total_len_seq'] = df.groupby(['sequence'],sort=False)['stop_pos'].transform(max)

# calculate percentage coverage for each mapping piece to the contig
df['percentage_of_seq'] = df['num_of_nucl'] / df['total_len_seq']


# make a new df where only values with coverage not 0 are in
df_no0 = df.loc[df['coverage'] != 0]

# add up all the percentages for sequences with the same name
# reset index to keep column names
df_percentages = df_no0.groupby(['sequence'])['percentage_of_seq'].agg('sum').reset_index()

# make it actual percentages by multiplying with 100
df_percentages['percentage_of_seq'] = df_percentages['percentage_of_seq']*100


# Select the cases where the percentage coverage is < 74.99
percentage_under_75 = df_percentages['percentage_of_seq'] < 74.99

# keep cases where perc coverage < 75, to new csv
df_percentages[percentage_under_75].to_csv(sys.argv[2], sep='\t')


## Use Bamm to make  a coverage table from all bam files
- http://ecogenomics.github.io/BamM/

In [8]:
# Use bamm to create coverage table
bamm parse -c output_file.tsv -b *.bam -m 'tpmean'

## Filter the coverage table based on the 75% lenght coverage parameter 
- This is done with custom python script
- Input is the coverage table and the SPRUCE_contigs_under75.csv file

In [ ]:
#! /usr/bin/env python 

# imports
import pandas as pd
import numpy as np
import re
import sys

# open the coverage table
df = pd.read_csv('coverage_table.tsv',sep='\t')

# open the under 75% coverage list per sample
df_under_75= pd.read_csv('under_75_per_sample.csv',sep='|')

# make the headers readable, cut from the _S# part
df.columns = df.columns.str.split('_L006').str[0] + ''
df_under_75.columns = df_under_75.columns.str.split('_L006').str[0] + ''

#remove viral contigs with all 0s from the coverage file
df_cov = remove_all_zeros(df)

# check the lenght of the dataframe and the number of non zero values in each 
print 'before', len(df_cov), df_cov.astype(bool).sum(axis=0)

#Do the function that checks each column(sample) for a coverage under 75%. 
for col in df_under_75.columns:
       df_cov[col] = df_cov[['#contig', col]].apply(lambda x: check_similarity(x['#contig'], df_under_75[col].values[1:], x[col]),axis=1)
    
#after removing these under 75% coverage ones, remove the columns with all 0 again.
df_cov = remove_all_zeros(df_cov)


# check the len of df and num of viral contigs with > 75% coverage for each sample
print 'after', len(df_cov),  df_cov.astype(bool).sum(axis=0)

#print df_cov
df_cov.to_csv('filtered_coverage_table.csv', sep='|')


def remove_all_zeros(df):

    # get a list of columns that could contain zero
    all_columns =  list(df.columns)[2:]

    # check if all columns contain zero
    df["all_zeros"] = (df[all_columns] == 0).all(axis=1)

    # remove all that have only zeros
    df = df.loc[df["all_zeros"] == False]

    # remove all zeros column
    df.drop(['all_zeros'], axis=1, inplace=True)
    
    return df

def check_similarity(contig_name, contig_names_to_check, curent_value):
        
    if contig_name in contig_names_to_check:
        return 0
    else: return curent_value
